# How to work with columnstore indexes
In previous demo we tried to optimize SalesOrder table for aggregate query using B-tree indexes. Now, we'll make a step ahead using columnstore index.

Let's recall the aggregate query shortly.

In [ ]:
use Demo
go
set statistics time, io ON
set statistics profile on

-- create index ix_SalesOrders_CustomerId on dbo.SalesOrders (CustomerId, Subtotal) with DROP_EXISTING
-- go

SELECT
    CustomerId
    , sum(Subtotal)
from SalesOrders
group by CustomerId
order by CustomerId
go

## Nonclustered columnstore  

Preceding query is covered by the index called ix\_SalesOrders\_CustomerId with couple of included column. This index eliminated the need to scan whole table. But what if user will execute a query using other grouping criteria? The index will not be used anymore. Let's try to create nonclustered columnstore index.

In [ ]:
use Demo
go

create columnstore index cs_SalesOrders on SalesOrders 
(
    Id
    , OrderDate
    , OrderNumber
    , CustomerId
    , Subtotal
    , OrderStatus
)
go

Now, when the index is created, we can test its performance with following queries. First query is the same as in previous example, Second query uses different gouping criteria.

In [ ]:
use Demo
go

SELECT
    CustomerId
    , sum(Subtotal)
from SalesOrders
group by CustomerId
order by CustomerId
go

SELECT
    CustomerId
    , OrderStatus
    , sum(Subtotal)
from SalesOrders
group by CustomerId, OrderStatus
order by CustomerId, OrderStatus
go

As we can see in results, SQL Server is very happy to use the columnstore index for aggregate queries now, and the performance is tens of times better than before when B-tree index was used. As a bonus, we can drop unused B-tree index. It will reduce waits when data is inserted or updated in base table.

In [ ]:
use Demo
go
drop index ix_SalesOrders_CustomerId on SalesOrders
go

## Clustered columnstore index
Clustered columnstore index is the best for DWH purposes. Let's create a copy of SalesOrders table, then we'll observer the space occupied by the copy, and as last step we'll create clustered columnstore there.

In [ ]:
use Demo
GO

set STATISTICS time, io OFF
set STATISTICS profile off

drop table if exists SalesOrdersCopy
select *
into SalesOrdersCopy
from SalesOrders
go

exec sp_spaceused 'SalesOrdersCopy'
go

In [ ]:
use Demo
go
create clustered columnstore index ccs_SalesOrdersCopy on SalesOrdersCopy
go
exec sp_spaceused 'SalesOrdersCopy'
go